In [1]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    #! mkdir -p ../data
    # los que usan colab deben modificar el token de esta url:
    #! wget -O ../data/properati_2017.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
Running command `conda install --yes chart-studio=1.1.0`... error
    
PackagesNotFoundError: The following packages are not available from current channels:

  - chart-studio=1.1.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.





In [2]:
import pandas as pd
import numpy as np
import re

In [109]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).
data     = pd.read_csv("./data/properati_2017.csv", sep = ",", low_memory=False)
geonames = pd.read_csv("./data/AR.txt", sep = "\t", header=None, low_memory=False)
geonames = geonames[[0, 4, 5]]
geonames.rename(columns={0:'geonames_id', 4: 'gn_lat', 5: 'gn_lon'}, inplace=True)

In [110]:
# Utilities
def drop_columns(df, columns):
    all_columns = set(df.columns)
    relevant_columns = all_columns.difference(set(columns))
    return df.loc[:, list(relevant_columns)]

def show_null_distribution(df):
    nulls = df.isnull().sum(axis=0)
    nulls_percent = nulls / df.index.size
    result = pd.concat([nulls, nulls_percent], axis=1)
    result.columns = ['total', 'percent']
    return result[result['total'] > 0]

In [112]:
 data.shape

(121220, 26)

In [113]:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [114]:
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [115]:
# categoricals
data["operation"]     = data["operation"].astype('category')
data["property_type"] = data["property_type"].astype('category')
data["country_name"]  = data["country_name"].astype('category')
data["state_name"]    = data["state_name"].astype('category')

In [116]:
# normalizations
regex = re.compile(r'\s+')
fformat = lambda x: regex.sub(' ', str(x)).strip().lower()
data['description'] = data['description'].apply(fformat)
data['title']       = data['title'].apply(fformat)

In [117]:
# NA - distribution
data = data.dropna(how='all')
show_null_distribution(data)

,total,percent
place_name,23,0.000190
geonames_id,18717,0.154405
lat-lon,51550,0.425260
lat,51550,0.425260
lon,51550,0.425260
price,20410,0.168372
currency,20411,0.168380
price_aprox_local_currency,20410,0.168372
price_aprox_usd,20410,0.168372
surface_total_in_m2,39328,0.324435


In [118]:
# NA - place_name
na_place_name_mask = data["place_name"].isnull()
na_place_name      = data[na_place_name_mask][["place_name", "place_with_parent_names"]]
na_place_name.drop_duplicates()

,place_name,place_with_parent_names
6489,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||


In [119]:
data['place_name'] = data['place_name'].fillna("Tigre").astype('category')

In [120]:
# NA - lat/lon
data = pd.merge(data, geonames, on='geonames_id', how='left', left_index=True)
data['lat'].fillna(data['gn_lat'])
data['lon'].fillna(data['gn_lon'])
data = drop_columns(data, ['gn_lat', 'gn_lon'])
show_null_distribution(data)

,total,percent
image_thumbnail,3112,0.025672
price_per_m2,33562,0.276869
surface_total_in_m2,39328,0.324435
lat-lon,51550,0.425260
rooms,73830,0.609058
floor,113321,0.934837
price,20410,0.168372
lat,51550,0.425260
price_aprox_local_currency,20410,0.168372
currency,20411,0.168380


In [129]:
# NA - price/currency
def get_price_from_title(text):
    regex = r'^\s*(\$|u\$d|usd)\s*([0-9\.,]+).+en\sventa'
    m = re.search(regex, text, re.I)
    if m is None:
        return np.nan, np.nan
    currency = 'ARS' if m.group(1) == '$' else 'USD'
    price    = np.float64(m.group(2).replace('.', '').replace(',', '.'))
    return currency, price

def get_price_from_description(text):
    if text is not np.nan:    
        regex = r'(vendo\scasa|valor\sde\sventa|valor:|vendo|valor|precio\ses\sde|precio\sde)\s+(\$|u\$d|usd)\s*([0-9\.,]+)'
        m = re.search(regex, text, re.I)
        if m is not None:
            currency = 'ARS' if m.group(2) == '$' else 'USD'
            price    = m.group(3).strip('.')
            if len(price) > 0:
                return currency, np.float64(price.replace('.', '').replace(',', '.'))
    return np.nan, np.nan

data['title_price'], data['title_currency']             = zip(*data['title'].map(get_price_from_title))
data['description_price'], data['description_currency'] = \
    zip(*data['description'].map(get_price_from_description))

data['title_price'].notna().sum(), data['title_currency'].notna().sum(), \
data['description_price'].notna().sum(), data['description_currency'].notna().sum()

(2619, 2619, 4971, 4971)

In [139]:
# NA - rooms
def get_rooms_from_text(text):
    if text is not np.nan:
        regex = r'([0-9]+)\s+(amb\s|amb\.|ambiente)'
        m = re.search(regex, text, re.I)
        if m is not None:
            return np.int(m.group(1))
    return np.nan

data['title_rooms']       = data['title'].map(get_rooms_from_text)
data['description_rooms'] = data['description'].map(get_rooms_from_text)

data['title_rooms'].notna().sum(), data['description_rooms'].notna().sum()

(20342, 37286)

In [140]:
# NA - floor
def get_floor_from_text(text):
    if text is not np.nan:
        if 'plante baja' in text:
            return 0
        regex = r'(1°|2°|3°|4°|5°|6°|2do|3er|primer|segundo|tercer)\spiso'
        m = re.search(regex, text, re.I)
        if m is not None:
            piso = m.group(1)
            if piso in ['1°', 'primer']:
                return 1
            elif piso in ['2°', '2do', 'segundo']:
                return 2
            elif piso in ['3°', '3er', 'tercer']:
                return 3
            else:
                return np.int(piso.replace('°', ''))
    return np.nan

data['title_floor']       = data['title'].map(get_floor_from_text)
data['description_floor'] = data['description'].map(get_floor_from_text)

data['title_floor'].notna().sum(), data['description_floor'].notna().sum()

(224, 7542)

In [153]:
# NA - m2 cubiertos
def get_coveredm2_from_text(text):
    if text is not np.nan:
        regex = [
            r'(\d[0-9\.,]+)\s+m2\s+cub',
            r'metros\s+cubiertos\s*:?(\d[0-9\.,]+)',
            r'cubierta\s*:?(\d[0-9\.,]+)\s+m2',
        ]
        for pattern in regex:
            m = re.search(pattern, text, re.I)
            if m is not None:
                sup = m.group(1)
                sup = sup.replace('.', '').replace(',', '.')
                return np.nan if sup == '' else np.int(float(sup))
    return np.nan

data['description_surface_m2_covered'] = data['description'].map(get_coveredm2_from_text)

data['description_surface_m2_covered'].notna().sum()

3611

In [150]:
# NA - m2 cubiertos
def get_totalm2_from_text(text):
    if text is not np.nan:
        regex = [
            r'(\d[0-9\.,]+)\s+m2\s+tot',
            r'metros\s+totales\s*:?(\d[0-9\.,]+)',
        ]
        for pattern in regex:
            m = re.search(pattern, text, re.I)
            if m is not None:
                sup = m.group(1)
                sup = sup.replace('.', '').replace(',', '.')
                return np.nan if sup == '' else np.int(float(sup))
    return np.nan

data['description_surface_m2_total'] = data['description'].map(get_totalm2_from_text)

data['description_surface_m2_total'].notna().sum()

1315